# Work Sample: Generation of Interactive Dahsboard Elements

### Author: Leopoldo Carbajal (leo.carbajalg@gmail.com)

## Summary:

Data collection and transformation are essential steps in the processes of **transforming data into actionable insights**. An equally important step is to **effectively communicate those insights to decision makers**, in order to create a **possitive impact in organizations, governments and societies**.

In this notebook we show how to **quickly analyze and report trends** in data by using **interactive dashboard elements**. For this exercise we use data from the [Mexican Government repository of SARS-CoV-2 patients data](https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico) (*datos abiertos de la Secretaría de Salud de México*). These data are collected by the Ministry of Health of Mexico and are used to track the development of the SARS-CoV-2 pandemic. Important insights can be drawn from these data that can be used to develop and deploy response mechanisms to mitigate the negative impact of SARS-CoV-2 in the country.

In the following, we will focus on producing **a single interactive chart** able to report the following daily numbers by state:

1. number of confirmed cases of SARS-CoV-2.
2. number of patients that required admission to the hospital for treatment.
3. number of deaths of confirmed SARS-CoV-2 patients.

## Implementation:

In the cell below, we start by loading all the necessary libraries.

In [2]:
# HTTP and other libraries
import time as t
from datetime import datetime

# Various libraries
import pandas as pd
import numpy as np
import os

pd.set_option('max_columns',500)
pd.set_option('max_rows',500)

# Style, display
from IPython.display import HTML, display, Image

# Plotting
import bokeh as bk

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import CustomJS, DateSlider
from bokeh.plotting import ColumnDataSource, figure, output_file, show
from bokeh.layouts import layout
from bokeh.models import (Button, CategoricalColorMapper, ColumnDataSource,
                          HoverTool, Label, SingleIntervalTicker, Slider)

from bokeh.palettes import Spectral6, Accent
from bokeh.plotting import figure

---

## Data Loading

We use the most recent available data as of today about **reported SARS-CoV-2 cases in Mexico**. These data are were downloaded directly from the [Mexican Government repository of SARS-CoV-2 patients data](https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico).

The patients data is made available as a table in CSV format, and the description of each field in the table are included in Microsoft Excel files, which are included in the download.


In the cell below, we load the patients data to a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [2]:
patient_data = pd.read_csv(filepath_or_buffer='200806COVID19MEXICO.csv', encoding = "ISO-8859-1")

display(HTML(patient_data.head().to_html()))

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-08-06,0cfcc3,2,4,25,1,25,25,1,1,2020-04-08,2020-04-03,9999-99-99,97,2,47,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
1,2020-08-06,1d67f9,2,4,9,1,9,9,7,1,2020-04-27,2020-04-24,9999-99-99,97,2,62,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97
2,2020-08-06,07d040,2,3,25,2,25,25,6,1,2020-06-02,2020-05-27,9999-99-99,97,2,33,1,97,2,2,2,2,2,1,2,2,2,2,2,1,1,99,MÃ©xico,99,97
3,2020-08-06,17241d,2,3,27,1,27,27,4,1,2020-07-02,2020-06-28,9999-99-99,97,2,18,1,2,2,2,2,2,2,2,2,2,2,2,2,1,1,99,MÃ©xico,99,97
4,2020-08-06,0276dd,2,4,9,1,9,9,7,1,2020-04-03,2020-04-02,9999-99-99,97,2,34,1,2,2,2,2,2,2,2,2,2,2,2,2,99,1,99,MÃ©xico,99,97


Similarly, we load the catalog/dictionary of fields that will be used below

In [5]:
df_state_catalog = pd.read_excel(io='diccionario_datos_covid19/Catalogos_0412.xlsx', sheet_name='Catalogo de ENTIDADES')

df_state_catalog = df_state_catalog.set_index(keys='CLAVE_ENTIDAD', drop=True)

display(HTML(df_state_catalog.head().to_html()))

,ENTIDAD_FEDERATIVA,ABREVIATURA
CLAVE_ENTIDAD,,
1,AGUASCALIENTES,AS
2,BAJA CALIFORNIA,BC
3,BAJA CALIFORNIA SUR,BS
4,CAMPECHE,CC
5,COAHUILA DE ZARAGOZA,CL


---

## Data Processing

From a quick inspection of the patients table, we identify the following fields that will be used for generating the interactive chart:

1. TIPO_PACIENTE: type of patient, 1 = Hospitalized, 2 = Ambulatory.
2. RESULTADO: whether the patient tested positive for SARS-CoV-2 (1) or not (2).
3. ID_REGISTRO: encrypted patient case ID.
4. FECHA_INGRESO: date when the patient case was reported.
5. FECHA_DEF: death date, when applicable.
5. ENTIDAD_UM: state reporting the patient case.

First, we filter patients data and only keep those that tested positive for SARS-CoV-2.

In [6]:
patient_data = patient_data[patient_data['RESULTADO'] == 1]

In the cell below, we generate a DataFrame with the following data for each patient testing positive for SARS-CoV-2:

1. DATE: date when the patient case was reported.
2. ID: encrypted patient case ID.
3. STATE: name of the state reporting the case.
4. HOSPITALIZED: boolean field for identifying hospitalized (True) patients and ambulatory (False) patients.
5. DECEASED: boolean field for identifying whether the patient passed away (True) or not (False).

In [8]:
df = pd.DataFrame(columns=['DATE', 'ID', 'STATE', 'HOSPITALIZED', 'DECEASED'])

df['DATE'] = pd.to_datetime(patient_data['FECHA_INGRESO'])

df['ID'] = patient_data['ID_REGISTRO']

df['STATE'] = patient_data['ENTIDAD_UM']
for state_id in df['STATE'].unique():
    df['STATE'] = df['STATE'].replace(to_replace=state_id, value=df_state_catalog.loc[state_id]['ENTIDAD_FEDERATIVA'])

df['HOSPITALIZED'] = patient_data['TIPO_PACIENTE']
df['HOSPITALIZED'] = df['HOSPITALIZED'].replace(to_replace=1, value=False)
df['HOSPITALIZED'] = df['HOSPITALIZED'].replace(to_replace=2, value=True)

df['DECEASED'] = patient_data['FECHA_DEF']
df['DECEASED'] = ~(df['DECEASED'] == '9999-99-99')

df = df.set_index(keys='DATE', drop=True)

display(HTML(df.head().to_html()))

,ID,STATE,HOSPITALIZED,DECEASED
DATE,,,,
2020-04-08,0cfcc3,SINALOA,False,False
2020-04-27,1d67f9,CIUDAD DE MÉXICO,False,False
2020-06-02,07d040,SINALOA,False,False
2020-07-02,17241d,TABASCO,False,False
2020-04-03,0276dd,CIUDAD DE MÉXICO,False,False


Next, we generate DataFrames with aggregated data, which will be used to feed the interactive chart. Importantly, **we pad with zeros the dates with no reported cases in a given state**.

In [9]:
dates_in_data = df.index

dates_in_report = pd.date_range(start=dates_in_data.min(), end=dates_in_data.max(), freq='D')

states = sorted( df['STATE'].unique() )

df_cases = pd.DataFrame(index=states, columns=dates_in_report)
df_deceased = pd.DataFrame(index=states, columns=dates_in_report)
df_hospitalized = pd.DataFrame(index=states, columns=dates_in_report)

for date in dates_in_report:
    if date in dates_in_data:
        df_ = df.loc[date]
        if (len(df_.shape) == 1):
            df_ = pd.DataFrame(df_.to_dict(), index=[df_.name], columns=df_.index.values)
        
        for state in states:
            df_cases.loc[state][date] = np.sum(df_['STATE'] == state)
            df_deceased.loc[state][date] = df_[df_['STATE'] == state]['DECEASED'].sum()
            df_hospitalized.loc[state][date] = df_[df_['STATE'] == state]['HOSPITALIZED'].sum()
    else:
        for state in states:
            df_cases.loc[state][date] = 0
            df_deceased.loc[state][date] = 0
            df_hospitalized.loc[state][date] = 0

In the cell below we show a sample of one of the generated DataFrames with aggregated data

In [10]:
display(HTML(df_cases.head().to_html()))

,2020-01-13 00:00:00,2020-01-14 00:00:00,2020-01-15 00:00:00,2020-01-16 00:00:00,2020-01-17 00:00:00,2020-01-18 00:00:00,2020-01-19 00:00:00,2020-01-20 00:00:00,2020-01-21 00:00:00,2020-01-22 00:00:00,2020-01-23 00:00:00,2020-01-24 00:00:00,2020-01-25 00:00:00,2020-01-26 00:00:00,2020-01-27 00:00:00,2020-01-28 00:00:00,2020-01-29 00:00:00,2020-01-30 00:00:00,2020-01-31 00:00:00,2020-02-01 00:00:00,2020-02-02 00:00:00,2020-02-03 00:00:00,2020-02-04 00:00:00,2020-02-05 00:00:00,2020-02-06 00:00:00,2020-02-07 00:00:00,2020-02-08 00:00:00,2020-02-09 00:00:00,2020-02-10 00:00:00,2020-02-11 00:00:00,2020-02-12 00:00:00,2020-02-13 00:00:00,2020-02-14 00:00:00,2020-02-15 00:00:00,2020-02-16 00:00:00,2020-02-17 00:00:00,2020-02-18 00:00:00,2020-02-19 00:00:00,2020-02-20 00:00:00,2020-02-21 00:00:00,2020-02-22 00:00:00,2020-02-23 00:00:00,2020-02-24 00:00:00,2020-02-25 00:00:00,2020-02-26 00:00:00,2020-02-27 00:00:00,2020-02-28 00:00:00,2020-02-29 00:00:00,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00,2020-03-04 00:00:00,2020-03-05 00:00:00,2020-03-06 00:00:00,2020-03-07 00:00:00,2020-03-08 00:00:00,2020-03-09 00:00:00,2020-03-10 00:00:00,2020-03-11 00:00:00,2020-03-12 00:00:00,2020-03-13 00:00:00,2020-03-14 00:00:00,2020-03-15 00:00:00,2020-03-16 00:00:00,2020-03-17 00:00:00,2020-03-18 00:00:00,2020-03-19 00:00:00,2020-03-20 00:00:00,2020-03-21 00:00:00,2020-03-22 00:00:00,2020-03-23 00:00:00,2020-03-24 00:00:00,2020-03-25 00:00:00,2020-03-26 00:00:00,2020-03-27 00:00:00,2020-03-28 00:00:00,2020-03-29 00:00:00,2020-03-30 00:00:00,2020-03-31 00:00:00,2020-04-01 00:00:00,2020-04-02 00:00:00,2020-04-03 00:00:00,2020-04-04 00:00:00,2020-04-05 00:00:00,2020-04-06 00:00:00,2020-04-07 00:00:00,2020-04-08 00:00:00,2020-04-09 00:00:00,2020-04-10 00:00:00,2020-04-11 00:00:00,2020-04-12 00:00:00,2020-04-13 00:00:00,2020-04-14 00:00:00,2020-04-15 00:00:00,2020-04-16 00:00:00,2020-04-17 00:00:00,2020-04-18 00:00:00,2020-04-19 00:00:00,2020-04-20 00:00:00,2020-04-21 00:00:00,2020-04-22 00:00:00,2020-04-23 00:00:00,2020-04-24 00:00:00,2020-04-25 00:00:00,2020-04-26 00:00:00,2020-04-27 00:00:00,2020-04-28 00:00:00,2020-04-29 00:00:00,2020-04-30 00:00:00,2020-05-01 00:00:00,2020-05-02 00:00:00,2020-05-03 00:00:00,2020-05-04 00:00:00,2020-05-05 00:00:00,2020-05-06 00:00:00,2020-05-07 00:00:00,2020-05-08 00:00:00,2020-05-09 00:00:00,2020-05-10 00:00:00,2020-05-11 00:00:00,2020-05-12 00:00:00,2020-05-13 00:00:00,2020-05-14 00:00:00,2020-05-15 00:00:00,2020-05-16 00:00:00,2020-05-17 00:00:00,2020-05-18 00:00:00,2020-05-19 00:00:00,2020-05-20 00:00:00,2020-05-21 00:00:00,2020-05-22 00:00:00,2020-05-23 00:00:00,2020-05-24 00:00:00,2020-05-25 00:00:00,2020-05-26 00:00:00,2020-05-27 00:00:00,2020-05-28 00:00:00,2020-05-29 00:00:00,2020-05-30 00:00:00,2020-05-31 00:00:00,2020-06-01 00:00:00,2020-06-02 00:00:00,2020-06-03 00:00:00,2020-06-04 00:00:00,2020-06-05 00:00:00,2020-06-06 00:00:00,2020-06-07 00:00:00,2020-06-08 00:00:00,2020-06-09 00:00:00,2020-06-10 00:00:00,2020-06-11 00:00:00,2020-06-12 00:00:00,2020-06-13 00:00:00,2020-06-14 00:00:00,2020-06-15 00:00:00,2020-06-16 00:00:00,2020-06-17 00:00:00,2020-06-18 00:00:00,2020-06-19 00:00:00,2020-06-20 00:00:00,2020-06-21 00:00:00,2020-06-22 00:00:00,2020-06-23 00:00:00,2020-06-24 00:00:00,2020-06-25 00:00:00,2020-06-26 00:00:00,2020-06-27 00:00:00,2020-06-28 00:00:00,2020-06-29 00:00:00,2020-06-30 00:00:00,2020-07-01 00:00:00,2020-07-02 00:00:00,2020-07-03 00:00:00,2020-07-04 00:00:00,2020-07-05 00:00:00,2020-07-06 00:00:00,2020-07-07 00:00:00,2020-07-08 00:00:00,2020-07-09 00:00:00,2020-07-10 00:00:00,2020-07-11 00:00:00,2020-07-12 00:00:00,2020-07-13 00:00:00,2020-07-14 00:00:00,2020-07-15 00:00:00,2020-07-16 00:00:00,2020-07-17 00:00:00,2020-07-18 00:00:00,2020-07-19 00:00:00,2020-07-20 00:00:00,2020-07-21 00:00:00,2020-07-22 00:00:00,2020-07-23 00:00:00,2020-07-24 00:00:00,2020-07-25 00:00:00,2020-07-26 00:00:00,2020-07-27 00:00:00,2020-07-28 00:00:00,2020-07-29 00:00:00,2020-07-30 00:00:00

As it can be seen from the sample above, the number of reported positive cases of SARS-CoV-2 increase as we move to recent days.

---

## Interactive Chart Implementation

We now move to the implementation of the interactive chart. For this, we use [Bokeh](https://docs.bokeh.org/en/latest/index.html), a python library for interactive visualization.

The first step for generating the chart is to generate a data source, or [Column Data Source (CDS)](https://docs.bokeh.org/en/latest/docs/user_guide/data.html) as it is called in Bokeh. This is done in the cell below.

In [35]:
data = dict()

dates_in_report_str = [date.strftime("%Y-%m-%d") for date in dates_in_report]

for date_str, date in zip(dates_in_report_str,dates_in_report):
    data[date] = dict()
    data[date]['STATE'] = df_cases.index.to_series().reset_index(drop=True)
    data[date]['CASES'] = df_cases[date].reset_index(drop=True)
    data[date]['DECEASED'] = df_deceased[date].reset_index(drop=True)
    
    data[date]['DECEASED_SIZE'] = df_deceased[date].reset_index(drop=True)
    data[date]['DECEASED_SIZE'] = 50.0*data[date]['DECEASED_SIZE']/( data[date]['DECEASED_SIZE'].max() + 1e-7 )
    data[date]['DECEASED_SIZE'][data[date]['DECEASED_SIZE'] < 15.0] = 15.0
    
    data[date]['HOSPITALIZED'] = df_hospitalized[date].reset_index(drop=True)

# We initialize the data source with a recent day of data for practical purposes of seeing the incremental setup of the chart
source = ColumnDataSource(data=data[dates_in_report[-10]])

Then, we generate a canvas for the chart, defining the limits and scale of the x-axis and y-axis, the title of the chart and size of the canvas.

In [36]:
x_range = (1,df_cases.max().max())
y_range = (0,df_hospitalized.max().max())

plot = figure(x_range=x_range, 
              y_range=y_range, 
              title='Daily SARS-CoV-2 Report - Mexico', 
              plot_height=200,
              x_axis_type='log',
              y_axis_type='linear'
             )

Now, we set the axis tickers and labels

In [37]:
plot.xaxis.ticker = [10, 100, 500, 1000, 1500, 2000]
plot.xaxis.axis_label = "Number of Reported Cases (Logarithmic Scale)"
plot.yaxis.axis_label = "Number of Hospitalized Patients"

In order to easily identify the date of analyzed data, we will show the date on the chart. For this, we use a label and add it to the chart canvas.

In [38]:
# Define label with date
label = Label(x=0.1*x_range[1], y=0.8*y_range[1], text=dates_in_report_str[-10], text_font_size='70px', text_color='#eeeeee')

# Add to canvas
plot.add_layout(label)

We will highlight those states with larger number of deaths due to SARS-CoV-2 using tones of red, and those with less deaths with tones of blue. For this, we use a [linear color map](https://docs.bokeh.org/en/latest/docs/reference/transform.html).

In [39]:
linear_cmap = bk.transform.linear_cmap(field_name='DECEASED', 
                                       palette=Spectral6, 
                                       low=0.0, 
                                       high=source.data['DECEASED'].max()
                                      )

Then, we generate a bubble chart with the above elements and show it in the cell below

In [40]:
circles = plot.circle(x='CASES',
                      y='HOSPITALIZED',
                      size='DECEASED_SIZE',
                      source=source,
                      fill_color=linear_cmap,
                      fill_alpha=0.7,
                      line_color='#7c7e71',
                      line_width=0.9,
                      line_alpha=0.7
                     )

output_file("images/first_chart.html", title="Non-interactive chart")

report_layout = layout([plot], sizing_mode='scale_width')

show(report_layout)

In [3]:
display(HTML('images/first_chart.html'))

The resulting chart shows the number of reported cases of SARS-CoV_2 on the x-axis and the number of hospitalized patients on the y-axis. Each filled circle represents a state, but still is not possible to identify them. 

In order to clearly identify what state is representing each circle, we will include a [Bokeh hover tool](https://docs.bokeh.org/en/latest/docs/reference/models/tools.html?highlight=hovertool#bokeh.models.tools.HoverTool), our first interactive element of the chart. This will show different data for each state as we hover the mouse over the circles. 

In the cell below, we include the hover tool and display the resulting chart.

In [42]:
plot.add_tools(HoverTool(tooltips=[('STATE','@STATE'), ('CASES','@CASES'), ('HOSPITALIZED','@HOSPITALIZED'), ('DEADTHS','@DECEASED')], 
                         show_arrow=False, 
                         point_policy='follow_mouse'
                        )
              )

output_file("images/chart_with_hover_tool.html", title="Chart with hover tool")

report_layout = layout([plot], sizing_mode='scale_width')

show(report_layout)

In [4]:
display(HTML('images/chart_with_hover_tool.html'))

From the chart above, we now clearly identify each state, and start to observe **relevant trends**. For example, **we see that Mexico City (CIUDAD DE MÉXICO) leads the number of reported cases on July 28th with 1041 cases, but NUEVO LEÓN (a northern state) has the larger number of deaths, despite having nearly a third (299) of reported cases of Mexico City**.

We now put all together in the cell below to generate the final interactive chart. A final addition to the chart is a [Bokeh slider](https://docs.bokeh.org/en/latest/docs/reference/models/widgets.sliders.html), which the final user can use to modify the date of data being displayed on the chart.

In [46]:
# We initialize the data source with the first day of data
source = ColumnDataSource(data=data[dates_in_report[0]])

x_range = (1,df_cases.max().max())
y_range = (0,df_hospitalized.max().max())

# Canvas
plot = figure(x_range=x_range, 
              y_range=y_range, 
              title='Daily SARS-CoV-2 Report - Mexico', 
              plot_height=200,
              x_axis_type='log',
              y_axis_type='linear'
             )

plot.xaxis.ticker = [10, 100, 500, 1000, 1500, 2000]
plot.xaxis.axis_label = "Number of Reported Cases (Logarithmic Scale)"
plot.yaxis.axis_label = "Number of Hospitalized Patients"

# Define label with date
label = Label(x=0.1*x_range[1], y=0.8*y_range[1], text=dates_in_report_str[-10], text_font_size='70px', text_color='#eeeeee')

# Add to canvas
plot.add_layout(label)

# Linear color map
linear_cmap = bk.transform.linear_cmap(field_name='DECEASED', 
                                       palette=Spectral6, 
                                       low=0.0, 
                                       high=source.data['DECEASED'].max()
                                      )

# Bubble plot
circles = plot.circle(x='CASES',
                      y='HOSPITALIZED',
                      size='DECEASED_SIZE',
                      source=source,
                      fill_color=linear_cmap,
                      fill_alpha=0.7,
                      line_color='#7c7e71',
                      line_width=0.9,
                      line_alpha=0.7
                     )

# Hover tool
plot.add_tools(HoverTool(tooltips=[('STATE','@STATE'), ('CASES','@CASES'), ('HOSPITALIZED','@HOSPITALIZED'), ('DEADTHS','@DECEASED')], 
                         show_arrow=False, 
                         point_policy='follow_mouse'
                        )
              )


# Callback function to interact with data source and update chart
def slider_update(attrname, old, new):
    date = pd.Timestamp(slider.value, unit='ms').round('D')
    label.text = date.strftime("%Y-%m-%d")
    source.data = data[date]
    circles.glyph.fill_color = linear_cmap = bk.transform.linear_cmap(field_name='DECEASED', 
                                       palette=Spectral6, 
                                       low=0.0, 
                                       high=source.data['DECEASED'].max()
                                      )
    

# Slider to interact with chart
slider = DateSlider(start=dates_in_report[0], end=dates_in_report[-1], value=dates_in_report[0], step=1, title="Date", show_value=True)
slider.on_change('value', slider_update)

report_layout = layout([[plot],[slider]], sizing_mode='scale_width')

curdoc().add_root(report_layout)
curdoc().title = "Daily SARS-CoV-2 Report - Mexico"

bk.plotting.output_notebook()

show(report_layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [5]:
display(HTML('images/final_chart.html'))

The final interactive chart above is a static HTML version of the final interactive chart, which when ran as a web service correclty updates the chart. 

The figures below show snapshots of the interactive chart running on a Bokeh server in `localhost` at `port 5100`, which is started in a terminal as follows:

`bokeh serve --port 5100 --show Interactive_Dashboard_Elements.ipynb`

![title](images/snapshot_1.png)

![title](images/snapshot_2.png)

![title](images/snapshot_3.png)

![title](images/snapshot_4.png)